# INTRODUCCION
El objetivo del proyecto es implementar un sistema en la Aduana de Paraguay que permita determinar el grado de coincidencia entre la declaración de valor del importador y el contenido de la imagen de cada factura correspondiente al despacho.


## tags examples

- **SOLD TO:** J & S SRL
- **DATE INV:** 06/06/2016
- **INVOICE No:** 1634828
- **TOTAL DUE:** $16.350,00
- telefono(resultado_ocr)
- web_mail(resultado_ocr)
- domicilio(resultado_ocr, domicilio_buscado)
- importe(resultado_ocr, importe_buscado, cod_moneda)
- marca(resultado_ocr, marca)


### Importamos las librerias 

In [1]:
from wand.image import Image as Img #convierte un pdf en una imagenes
from PIL import Image #se utiliza para manipular las imagenes generadas
import pytesseract # lector OCR

import re #para trabajar con expresiones regulares
import json #exportamos nuestra información en un json

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' #define la ruta donde esta installado el tesseract

### 1.Covertimos el pdf a una imagen 

In [2]:
with Img(filename=r'raw\factura_001.pdf', resolution=300) as img:
    img.compression_quality = 99
    img
    img.save(filename='./raw/factura_001.jpg')

### 2. Extraemos el texto de la factura

In [3]:
text = pytesseract.image_to_string(Image.open('./raw/factura_001.jpg'))

In [4]:
print(text)

BROWNE FACTURA ELECTRONICA

INVOICE No.: 1989
AGENCIA DE ADUANAS

SOLD TO: MERCADO DE LAS FRUTAS DATE INV.: 09/12/2006

129.200

FACTURA DE GASTOS

DESEMBOLSOS CUENTA CLUENTE

Aduana Cancelado
Desembolso Cuenta Cliente

Factura de gastos Chile
Agencia de Aduanas 1

TOTAL DUE: 1.596.097


In [5]:
#archivo-salida.py
f = open ('./text/factura_001.txt','w')
f.write(text)
f.close()

## 3. Extraemos los datos y los integramos en un diccionario

In [6]:
dic_extracted = {'INVOICE' : re.findall(r'INVOICE No.: (\d+)', text),
                 'DATE' : re.findall(r'INV.: (\d+[/]\d+[/]\d+)', text),
                 'TOTAL DUE' : re.findall(r'TOTAL DUE: (\d+[.,]\d+[.,]\d+)', text)}
dic_extracted

{'INVOICE': ['1989'], 'DATE': ['09/12/2006'], 'TOTAL DUE': ['1.596.097']}

### 4. Validamos

####  6. Exportamos

In [7]:
with open('./json/factura_001.txt', 'w') as json_file:
  json.dump(dic_extracted, json_file)